In [ ]:
import numpy as np
import polars as pl

In [8]:
df = pl.read_csv("future_position_descriptions.csv")
df.head(1)

ID,Description
str,str
"""00066be8e20318869c38c66be46663…","""The center and right portions …"


In [ ]:
# tfidf -> svd
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform(df["Description"].to_list())
svd = TruncatedSVD(n_components=32)
X = svd.fit_transform(X)

In [5]:
X.shape

(45098, 32)

In [9]:
vlm_df = pl.DataFrame(X, schema=[f"vlm_svd_{i}" for i in range(32)])
vlm_df = vlm_df.with_columns(pl.Series("ID", df["ID"]))

In [10]:
vlm_df.head(1)

vlm_svd_0,vlm_svd_1,vlm_svd_2,vlm_svd_3,vlm_svd_4,vlm_svd_5,vlm_svd_6,vlm_svd_7,vlm_svd_8,vlm_svd_9,vlm_svd_10,vlm_svd_11,vlm_svd_12,vlm_svd_13,vlm_svd_14,vlm_svd_15,vlm_svd_16,vlm_svd_17,vlm_svd_18,vlm_svd_19,vlm_svd_20,vlm_svd_21,vlm_svd_22,vlm_svd_23,vlm_svd_24,vlm_svd_25,vlm_svd_26,vlm_svd_27,vlm_svd_28,vlm_svd_29,vlm_svd_30,vlm_svd_31,ID
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
0.354436,0.005188,0.005424,-0.170833,0.05523,0.088377,-0.018378,-0.019379,0.010477,-0.048484,-0.123945,-0.008,-0.052822,-0.01359,-0.01253,-0.050051,0.031472,-0.009266,-0.000254,-0.067036,0.024151,0.061267,-0.026678,-0.053421,0.004611,-0.042295,-0.042563,0.118235,-0.016544,-0.024801,0.025701,-0.018872,"""00066be8e20318869c38c66be46663…"


In [11]:
vlm_df.write_csv("vlm_svd.csv")